In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH12-RAG")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 문서 로드
loader = PyMuPDFLoader("12-RAG/data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 문서 분할
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50)
split_documents = text_spliter.split_documents(docs)


# 임베딩
embeddings = OpenAIEmbeddings()

# DB 생성 및 저장
# 벡터 스토어 생성
vectorstore = FAISS.from_documents(
    documents=split_documents, embedding=embeddings)

# 문서에 포함된 정보를 검색하고 생성한다.
retriver = vectorstore.as_retriever()

# 프롬프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

# 언어 모델 생성
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

chain = (
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "삼성전자가 자체 개발한 AI의 이름은?"
response = chain.invoke(question)
print(response)



삼성전자가 자체 개발한 AI의 이름은 '삼성 가우스'입니다.


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 문서 로드
loader = PyMuPDFLoader("doc1.pdf")
docs = loader.load()

# 문서 분할
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=10000, chunk_overlap=50)
split_documents = text_spliter.split_documents(docs)

# 임베딩
embeddings = OpenAIEmbeddings()

# DB 생성
vectorstore = FAISS.from_documents(
    documents=split_documents, embedding=embeddings)

# 문서에 포함된 정보를 검색하고 생성한다.
retriver = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
        """
        You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.
    
    #Question:
    {question}
    #Context:
    {context}
    
    #Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

chain=(
    {"context": retriver, "question":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

question = "z세대 청소년의 가치관은 어떻게 변화했습니까?"
response = chain.invoke(question)
print(response)

Z세대 청소년의 가치관은 급격한 사회 변화와 발전 속에서 자아정체감과 가치관이 확고하게 형성되지 않은 상태에서 새로운 의식구조와 행동양태를 형성하고 있습니다. 이들은 강한 독립심, 발전지향적 태도, 성취지향적 태도, 비판적 사회의식, 자유지향적 가치존중 등의 특징을 보이며, 이러한 가치관은 여전히 사회적으로 바람직한 것으로 평가받고 있습니다. 또한, 청소년에 대한 긍정적 접근이 필요하며, 청소년을 문제의 대상으로 보는 시각을 극복하고 그들의 긍정적 측면과 강점을 신뢰하는 것이 중요합니다.
